<a href="https://colab.research.google.com/github/dodeeric/langchain-ai-assistant-with-hybrid-rag/blob/main/BMAE_AI_Assistant_with_hybrid_RAG_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Assistant (LLM Chatbot) with Hybrid RAG
Hybrid RAG: keyword search (bm25) and semantic search (vector db)

In [ ]:
!pip install --upgrade --quiet jq bs4 langchain langchain-community langchain-openai langchain-chroma langchainhub rank_bm25

import requests, json, jq, time, bs4
from bs4 import BeautifulSoup
from google.colab import userdata
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, JSONLoader
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY") # To access OpenAI LLM and embedding model via API
LANGCHAIN_API_KEY = userdata.get("LANGCHAIN_API_KEY") # To trace Langchain on Langsmith

%env OPENAI_API_KEY = $OPENAI_API_KEY
%env LANGCHAIN_API_KEY = $LANGCHAIN_API_KEY
%env LANGCHAIN_TRACING_V2 = "true"

# import dotenv
# dotenv.load_dotenv()

## Scrape

In [ ]:
# Function to scrape the text and the metadata of a web page

def scrape_web_page(url):
    """
    Name: swp
    Scrape the text and the metadata of a web page
    Input: URL of the page
    Output: list of dictionaries with: url: url, metadata: metadata, text: text
    """

    #filter = "two-third last" # balat / irpa
    #filter = "notice_corps media" #  belgica / kbr
    filter = "hproduct commons-file-information-table" # commons / wikimedia: summary or description section

    # Get the page content
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=(filter)
            )
        ),
    )
    text = loader.load()
    # Covert Document type into string type
    text = text[0].page_content

    # Get the metadata (open graph from Facebook, og:xxx)
    # Get the HTML code
    response = requests.get(url)
    # Transform the HTML code from a Response object type into a BeautifulSoup object type to be scraped by Beautiful Soup
    soup = BeautifulSoup(response.text, "html.parser")
    # Get the metadata fields
    metadata = {} # Empty dictionary
    # Find all the meta tags in the HTML
    meta_tags = soup.find_all("meta")
    # Loop through the meta tags
    for tag in meta_tags:
        property = tag.get("property")
        content = tag.get("content")
        # Add the property-content pair to the dictionary
        if property and content:
            metadata[property] = content

    # Build JSON string with: url: url, metadata: metadata, text: summary text
    # Create a dictionary
    page = {
        "url": url, # String
        "metadata": metadata, # Dictionary
        "text": text # String
    }

    return page # Dictionary

In [ ]:
# Scrape the URLs and save the results in a JSON file

file_path = "/content/drive/MyDrive/colab/commons-urls-ds1"
#file_path = "/content/drive/MyDrive/colab/commons-urls-test"
#file_path = "/content/drive/MyDrive/colab/balat-urls-test"
#file_path = "/content/drive/MyDrive/colab/belgica-urls-test"

with open(f"{file_path}.txt", "r") as urls_file:
    items = []
    for line in urls_file:
        url = line.strip()
        url = url.replace("\ufeff", "")  # Remove BOM
        item = scrape_web_page(url)
        print(item)
        items.append(item)
        time.sleep(1)

# Save the Python list in a JSON file
# json.dump is designed to take the Python objects, not the already-JSONified string. Read docs.python.org/3/library/json.html.
with open(f"{file_path}-swp.json", "w") as json_file:
    json.dump(items, json_file) # Replaces the accentuated characters (ex: é) by its utf8 codes (ex: \u00e9)
json_file.close()

In [ ]:
# Open the JSON file to check its content (will produce an error if it's not a correctly formated JSON file)
with open(f"{file_path}-swp.json", "r") as input_file:
    items_read = json.load(input_file)

## Index

In [3]:
# Open the JSON files and load each JSON item one by one in the "documents" variable (type: Document)

file_path1 = "/content/drive/MyDrive/colab/commons-urls-ds1-swp.json"
file_path2 = "/content/drive/MyDrive/colab/balat-ds1c-wcc-cheerio-ex_2024-04-06_09-05-15-262.json"
file_path3 = "/content/drive/MyDrive/colab/belgica-ds1c-wcc-cheerio-ex_2024-04-06_08-30-26-786.json"
file_paths = [file_path1, file_path2, file_path3]

documents = []
for file_path in file_paths:
    loader = JSONLoader(file_path=file_path, jq_schema=".[]", text_content=False)
    docs = loader.load() # Chunks (JSON items) from the JSON files; list of Documents
    documents = documents + docs # This variable contents all the JSON items

Run step 1 or step 2:

In [4]:
# STEP 1: Instanciate a Chroma DB and load the data from disk.
collection_name = "bmae-json"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large") # 3072 dimensions vectors used to embed the JSON items and the questions
vector_db = Chroma(embedding_function=embedding_model, collection_name=collection_name, persist_directory="/content/drive/MyDrive/colab/chromadb2")

In [ ]:
# STEP 2: ONLY TO EMBED! Instantiate a Chroma DB, embed the JSON items (documents), then save to disk.
collection_name = "bmae-json"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large") # 3072 dimensions vectors used to embed the JSON items and the questions
vector_db = Chroma.from_documents(documents, embedding_model, collection_name=collection_name, persist_directory="/content/drive/MyDrive/colab/chromadb2")
# To check the Chroma vector db (sqlite3):
# $ sqlite3 chroma.sqlite3
# sqlite> .tables ===> List of the tables
# sqlite> select * from collections; ===> Name of the collection & size of the vectors
# sqlite> select * from embeddings; ===> Number of records in the db
# sqlite> select * from embedding_metadata; ===> Display json items

## Retrieve and generate

In [5]:
# LLM chatbot with a hybrid RAG chain:
# (To embed the question, the same model is used as for the data; the model is given in "vector_db".)

llm = ChatOpenAI(model="gpt-4-turbo-2024-04-09", temperature=0)

# Semantic search (vector retriever)
vector_retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # Chroma DB

# Keyword search (bm25 retriever)
keyword_retriever = BM25Retriever.from_documents(documents)
keyword_retriever.k = 3

# Ensemble retriever (mix of both retrivers) -- Weights = order of the results!!! [1,0] means: all bm25 first, all vector after...
ensemble_retriever = EnsembleRetriever(retrievers=[keyword_retriever, vector_retriever], weights=[0.5, 0.5])

# Download prompt template (system prompt + context (rag documents) + user question)
prompt = hub.pull("dodeeric/rag-prompt-bmae")

# Take the text content of each doc, and concatenate them in one string to pass to the prompt (context)
def format_docs_clear_text(docs):
    return "\n\n".join(doc.page_content.encode('utf-8').decode('unicode_escape') for doc in docs)

# Function to display the text content of the prompt in ai_assistant_chain
def print_and_pass(data):
    print(f"Prompt content sent to the LLM: {data}")
    return data

# Langchain chain: the LLM chatbot with hybrid RAG:
ai_assistant_chain = (
    {"context": ensemble_retriever | format_docs_clear_text, "question": RunnablePassthrough()}
    | prompt
    #| print_and_pass
    | llm
    | StrOutputParser() # Convert to string
)

Query the AI Assistant:

In [6]:
question = "Can you show me the canvas 'the review of the schools' which is present in the BALaT KIK-IRPA database, not the one from Wikimedia Commons?"

In [7]:
ai_assistant_chain.invoke(question) # Output is in markdown format

'Certainly! The painting "La revue des écoles en 1878" by Jan Frans Verhas is present in the BALaT KIK-IRPA database. This artwork depicts the review of the communal schools of the city of Brussels, which took place on August 23, 1878, in front of the Royal Palace of Brussels. The event was held in honor of the silver wedding anniversary of King Leopold II and Queen Marie-Henriette. The painting captures a significant moment with various dignitaries and members of the royal family, including King Leopold II, present during the review.\n\nFor more details, you can visit the BALaT KIK-IRPA page for this artwork:\n[La revue des écoles en 1878 - BALaT KIK-IRPA](https://balat.kikirpa.be/object/130731)\n\n![La revue des écoles en 1878](http://balat.kikirpa.be/image/thumbnail/B213530.jpg)'

In [ ]:
# Query the vector RAG only
docs = vector_db.similarity_search(question, k=2) # List of Documents; page_content of a Document: string
rag_context = format_docs_clear_text(docs) # One string composed of k json items
print(rag_context)

# Streamlit Web interface

In [ ]:
!pip install streamlit

In [ ]:
#!pip install localtunnel

In [ ]:
# The IP is the password
!wget -q -O - ipv4.icanhazip.com

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd

spectra = st.file_uploader("upload CSV file", type={"csv"})
if spectra is not None:
  spectra_df = pd.read_csv(spectra)
  st.write(spectra_df)

In [ ]:
%%writefile app.py

import streamlit as st

st.title('AI-powered Document Query Assistant')

# Text input for the query
user_query = st.text_area("Enter your query:", help='Type your query here and press enter.')

# Button to submit query
if st.button('Search'):
    if user_query:
        # This is where your AI assistant's processing happens
        response = ai_assistant_chain.invoke(user_query)
        st.write(response)  # Display the response
    else:
        st.write("Please enter a query to proceed.")

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501